In [1]:
%reset
import numpy as np
import healpy as hp
import sys
from scipy import interpolate
import os
import pdb
import kmeans_radec
import pickle as pk
from astropy.io import fits




Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [15]:
from scipy.interpolate import *
import numpy as np

omegab = 0.049
omegac = 0.261
omegam = omegab + omegac
h      = 0.68
ns     = 0.965
sigma8 = 0.81

c = 3e5

H0 = 100*h
nz = 100000
z1 = 0.0
z2 = 6.0
za = np.linspace(z1,z2,nz)
dz = za[1]-za[0]

H      = lambda z: H0*np.sqrt(omegam*(1+z)**3+1-omegam)
dchidz = lambda z: c/H(z)

chia = np.cumsum(dchidz(za))*dz

zofchi = interp1d(chia,za)

chiofz = interp1d(za,chia)



In [21]:
import healpy as hp
from   cosmology import *

rho = 2.775e11*omegam*h**2 # Msun/Mpc^3

f=open('/global/project/projectdirs/des/shivamp/actxdes/data_set/cita_sims/halos-light.pksc')
N=np.fromfile(f,count=3,dtype=np.int32)[0]

# only take first five entries for testing (there are ~8e8 halos total...)
# comment the following line to read in all halos
N = 1000000

catalog=np.fromfile(f,count=N*10,dtype=np.float32)
catalog=np.reshape(catalog,(N,10))

x  = catalog[:,0];  y = catalog[:,1];  z = catalog[:,2] # Mpc (comoving)
vx = catalog[:,3]; vy = catalog[:,4]; vz = catalog[:,5] # km/sec
R  = catalog[:,6] # Mpc

# convert to mass, comoving distance, radial velocity, redshfit, RA and DEc
M200m    = 4*np.pi/3.*rho*R**3        # this is M200m (mean density 200 times mean) in Msun
chi      = np.sqrt(x**2+y**2+z**2)    # Mpc
vrad     = (x*vx + y*vy + z*vz) / chi # km/sec
redshift = zofchi(chi)




In [19]:
catalog[3,:]



array([-8.7495624e+02,  4.4539118e+00, -7.3393867e+03, -5.0009705e+02,
       -8.4370898e+02,  3.8154209e+00, -7.3436382e+03, -5.7742072e+02,
       -8.3129315e+02,  3.8590510e+00], dtype=float32)

In [16]:
halo_file = '/global/project/projectdirs/des/shivamp/actxdes/data_set/cita_sims/halos-light.pksc'
pkfile = open(halo_file, "rb")
Nhalo=100
# Nhalo = np.fromfile(pkfile, dtype=np.int64, count=1)[0]
# RTHMAXin = np.fromfile(pkfile, dtype=np.float64, count=1)
# redshiftbox = np.fromfile(pkfile, dtype=np.float64, count=1)
print "\nNhalo = ", Nhalo

nfloats_perhalo = 10
npkdata = nfloats_perhalo * Nhalo
peakdata = np.fromfile(pkfile, dtype=np.float64, count=npkdata)
peakdata = np.reshape(peakdata, (Nhalo, nfloats_perhalo))

xpos = peakdata[:, 0]
ypos = peakdata[:, 1]
zpos = peakdata[:, 2]
vxpos = peakdata[:, 3]
vypos = peakdata[:, 4]
vzpos = peakdata[:, 5]
Rth = peakdata[:, 6]
Omega_M = 0.31
h = 0.68
rho = 2.775e11 * Omega_M * h ** 2
M = 4.0 / 3 * np.pi * Rth ** 3 * rho
r = np.sqrt(xpos ** 2. + ypos ** 2. + zpos ** 2.)
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from scipy import interpolate as intrp
cosmo = FlatLambdaCDM(H0=h * 100, Om0=Omega_M)
z_array = np.linspace(0., 6.0, num=3000)
r_interp = cosmo.comoving_distance(z_array)
z_interp_func = intrp.interp1d(r_interp.value, z_array)
z = z_interp_func(r)





Nhalo =  100


ValueError: A value in x_new is above the interpolation range.

In [16]:

hcato = fits.open('/global/project/projectdirs/des/shivamp/actxdes/data_set/cita_sims/halos_ds_10.fits')



In [17]:
M200m = hcato[1].data['M']
red = hcato[1].data['Z']
ra = hcato[1].data['RA']
dec = hcato[1].data['DEC']



In [18]:
def ang2eq(theta, phi):
    ra = phi * 180. / np.pi
    dec = 90. - theta * 180. / np.pi
    return ra, dec


def eq2ang(ra, dec):
    phi = ra * np.pi / 180.
    theta = (np.pi / 2.) - dec * (np.pi / 180.)
    return theta, phi




In [19]:
theta, phi = eq2ang(ra, dec)




In [20]:
vec_all = hp.ang2vec(theta,phi)




In [21]:

x,y,z = vec_all[:,0],vec_all[:,1],vec_all[:,2]




In [22]:
chi = chiofz(red)




In [26]:
xt,yt,zt = x*chi, y*chi, z*chi



In [27]:
c1 = fits.Column(name='PX', array=xt, format='E')
c2 = fits.Column(name='PY', array=yt, format='E')
c3 = fits.Column(name='PZ', array=zt, format='E')
c4 = fits.Column(name='M200m', array=M200m, format='E')
t = fits.BinTableHDU.from_columns([c1, c2, c3, c4])
t.writeto('/global/project/projectdirs/des/shivamp/actxdes/data_set/cita_sims/halos_light_ds_10_xyz.fits', clobber=True)



